<a href="https://colab.research.google.com/github/Hemang-sen/UCS551_SummerSem_HemangSen-102216064-/blob/main/ASSIGNMENT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
file_path = "/content/drive/MyDrive/price_paid_records.csv"

### QUESTION 1

In [2]:
import pandas as pd
import cudf
import time
import numpy as np
start_time = time.time()
pdf = pd.read_csv(file_path)
pandas_load_time = time.time() - start_time
print(f"Pandas load time: {pandas_load_time:.4f} seconds")

start_time = time.time()
gdf = cudf.read_csv(file_path)
cudf_load_time = time.time() - start_time
print(f"cuDF load time: {cudf_load_time:.4f} seconds")

Pandas load time: 68.1578 seconds
cuDF load time: 14.7472 seconds


In [3]:
start_time = time.time()
print("Pandas .info():")
pdf.info()
print("\nPandas .describe():")
print(pdf.describe())
pandas_info_desc_time = time.time() - start_time
print(f"Pandas .info() + .describe(): {pandas_info_desc_time:.4f} seconds")


start_time = time.time()
print("\ncuDF .info():")
gdf.info()
print("\ncuDF .describe():")
print(gdf.describe())
cudf_info_desc_time = time.time() - start_time
print(f"cuDF .info() + .describe(): {cudf_info_desc_time:.4f} seconds")

Pandas .info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22489348 entries, 0 to 22489347
Data columns (total 11 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   Transaction unique identifier      object
 1   Price                              int64 
 2   Date of Transfer                   object
 3   Property Type                      object
 4   Old/New                            object
 5   Duration                           object
 6   Town/City                          object
 7   District                           object
 8   County                             object
 9   PPDCategory Type                   object
 10  Record Status - monthly file only  object
dtypes: int64(1), object(10)
memory usage: 1.8+ GB

Pandas .describe():
              Price
count  2.248935e+07
mean   1.782442e+05
std    3.903677e+05
min    1.000000e+00
25%    7.500000e+04
50%    1.300000e+05
75%    2.100000e+05
max    9.890000e+07
Pandas

In [4]:
start_time = time.time()
pdf_cleaned = pdf.dropna()
pandas_dropna_time = time.time() - start_time
print(f"Pandas dropna: {pandas_dropna_time:.4f} seconds")


start_time = time.time()
gdf_cleaned = gdf.dropna()
cudf_dropna_time = time.time() - start_time
print(f"cuDF dropna: {cudf_dropna_time:.4f} seconds")

Pandas dropna: 12.6697 seconds
cuDF dropna: 0.0418 seconds


In [5]:
pdf_cleaned.columns = pdf_cleaned.columns.str.strip()
gdf_cleaned.columns = gdf_cleaned.columns.str.strip()

print("Pandas Columns:\n", pdf_cleaned.columns.tolist())
print("cuDF Columns:\n", gdf_cleaned.columns.tolist())

Pandas Columns:
 ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']
cuDF Columns:
 ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']


In [7]:
pdf_cleaned.columns = pdf_cleaned.columns.str.strip()
gdf_cleaned.columns = gdf_cleaned.columns.str.strip()

print("📌 Columns in pandas DataFrame:")
print(pdf_cleaned.columns.tolist())

📌 Columns in pandas DataFrame:
['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']


In [9]:
import numpy as np
import cupy as cp
import time

pdf_cleaned.columns = pdf_cleaned.columns.str.strip()
gdf_cleaned.columns = gdf_cleaned.columns.str.strip()

start_time = time.time()
pdf_cleaned['log_price'] = np.log(pdf_cleaned['Price'])
pandas_log_time = time.time() - start_time
print(f"Pandas log(Price): {pandas_log_time:.4f} seconds")

start_time = time.time()
gdf_cleaned['log_price'] = cp.log(gdf_cleaned['Price'].to_cupy())
cudf_log_time = time.time() - start_time
print(f"cuDF log(Price): {cudf_log_time:.4f} seconds")
print(pdf_cleaned.columns.tolist())

Pandas log(Price): 0.2817 seconds
cuDF log(Price): 0.0219 seconds
['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only', 'log_price']


In [12]:
from datetime import datetime, timedelta
date_column = 'Date of Transfer'
cutoff_date = datetime.now() - timedelta(days=5*365)
gdf_cleaned = gdf_cleaned.dropna(subset=[date_column])
gdf_cleaned[date_column] = cudf.to_datetime(gdf_cleaned[date_column])
start_time = time.time()
gdf_recent = gdf_cleaned[gdf_cleaned[date_column] >= cutoff_date]
cudf_filter_time = time.time() - start_time
print(f"cuDF filter (last 5 years): {cudf_filter_time:.4f} seconds")

cuDF filter (last 5 years): 0.0302 seconds


In [13]:
gdf_cleaned = gdf_cleaned.dropna(subset=[date_column])
gdf_cleaned[date_column] = cudf.to_datetime(gdf_cleaned[date_column])
start_time = time.time()
gdf_recent = gdf_cleaned[gdf_cleaned[date_column] >= cutoff_date]
cudf_filter_time = time.time() - start_time
print(f"cuDF filter (last 5 years): {cudf_filter_time:.4f} seconds")

cuDF filter (last 5 years): 0.0042 seconds


### QUESTION 2

In [14]:
import pandas as pd
import numpy as np
import time
pdf.columns = pdf.columns.str.strip()
pdf_cleaned = pdf.dropna()
pdf_cleaned['Price'] = pdf_cleaned['Price'].astype(float)  # Ensure numeric
pdf_cleaned['log_price'] = np.log(pdf_cleaned['Price'])

In [15]:
print(pdf_cleaned.columns.tolist())
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Property Type', 'Town/City', 'County']
label_encoders = {}

for col in categorical_cols:
    pdf_cleaned[col] = pdf_cleaned[col].astype(str).str.strip()
    le = LabelEncoder()
    pdf_cleaned[col] = le.fit_transform(pdf_cleaned[col])
    label_encoders[col] = le

['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only', 'log_price']


In [16]:
median_price = pdf_cleaned['Price'].median()
pdf_cleaned['price_category'] = (pdf_cleaned['Price'] > median_price).astype(int)

In [17]:
from sklearn.model_selection import train_test_split
features = ['Property Type', 'Town/City', 'County', 'log_price']
X = pdf_cleaned[features]
y = pdf_cleaned['price_category']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [18]:
sampled_df = pdf_cleaned.sample(n=50000, random_state=42)
features = ['Property Type', 'Town/City', 'County', 'log_price']
X = sampled_df[features]
y = sampled_df['price_category']

In [19]:
from sklearn.model_selection import train_test_split
features = ['Property Type', 'Town/City', 'County', 'log_price']
X = sampled_df[features]
y = sampled_df['price_category']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import time
start_time = time.time()
rf_sklearn = RandomForestClassifier(n_estimators=20, max_depth=10, random_state=42)
rf_sklearn.fit(X_train, y_train)
sklearn_train_time = time.time() - start_time
start_time = time.time()
y_pred_sklearn = rf_sklearn.predict(X_test)
sklearn_predict_time = time.time() - start_time
sklearn_acc = accuracy_score(y_test, y_pred_sklearn)
sklearn_f1 = f1_score(y_test, y_pred_sklearn)
print("Scikit completed")

Scikit completed


In [22]:
from cuml.model_selection import train_test_split as cu_train_test_split
from cuml.ensemble import RandomForestClassifier as cuRF
import cudf
from sklearn.metrics import accuracy_score, f1_score
import time
if not isinstance(sampled_df, cudf.DataFrame):
    gdf = cudf.from_pandas(sampled_df)
else:
    gdf = sampled_df

features = ['Property Type', 'Town/City', 'County', 'log_price']
X_cu = gdf[features]
y_cu = gdf['price_category']
X_train_cu, X_test_cu, y_train_cu, y_test_cu = cu_train_test_split(
    X_cu, y_cu, test_size=0.2, random_state=42)
start_time = time.time()
rf_cuml = cuRF(n_estimators=20, max_depth=10, random_state=42)
rf_cuml.fit(X_train_cu, y_train_cu)
cuml_train_time = time.time() - start_time
print(f"cuML training time: {cuml_train_time:.4f} seconds")
start_time = time.time()
y_pred_cuml = rf_cuml.predict(X_test_cu)
cuml_predict_time = time.time() - start_time
print(f"cuML prediction time: {cuml_predict_time:.4f} seconds")
cuml_acc = accuracy_score(y_test_cu.to_numpy(), y_pred_cuml.to_numpy())
cuml_f1 = f1_score(y_test_cu.to_numpy(), y_pred_cuml.to_numpy())
print("\ncuML completed")
print(f"cuML Accuracy: {cuml_acc:.4f}")
print(f"cuML F1 Score: {cuml_f1:.4f}")

print("\nComparison:")
print(f"Scikit-learn Training Time: {sklearn_train_time:.4f} seconds")
print(f"cuML Training Time: {cuml_train_time:.4f} seconds")
print(f"Scikit-learn Prediction Time: {sklearn_predict_time:.4f} seconds")
print(f"cuML Prediction Time: {cuml_predict_time:.4f} seconds")
print(f"Scikit-learn Accuracy: {sklearn_acc:.4f}")
print(f"cuML Accuracy: {cuml_acc:.4f}")
print(f"Scikit-learn F1 Score: {sklearn_f1:.4f}")
print(f"cuML F1 Score: {cuml_f1:.4f}")

/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:368: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return init_func(self, *args, **kwargs)


cuML training time: 2.7424 seconds
cuML prediction time: 0.2353 seconds

cuML completed
cuML Accuracy: 1.0000
cuML F1 Score: 1.0000

Comparison:
Scikit-learn Training Time: 0.2141 seconds
cuML Training Time: 2.7424 seconds
Scikit-learn Prediction Time: 0.0179 seconds
cuML Prediction Time: 0.2353 seconds
Scikit-learn Accuracy: 1.0000
cuML Accuracy: 1.0000
Scikit-learn F1 Score: 1.0000
cuML F1 Score: 1.0000


### QUESTION 3

In [23]:
pdf.columns = pdf.columns.str.strip()
pdf_cleaned = pdf.dropna()
pdf_cleaned['Price'] = pdf_cleaned['Price'].astype(float)
pdf_cleaned['log_price'] = np.log(pdf_cleaned['Price'])

In [24]:
from itertools import combinations
sampled_df = pdf_cleaned.sample(n=50000, random_state=42)
df = sampled_df.copy()
df['postcode'] = df['Transaction unique identifier'].str.extract(r'([A-Z]{1,2}\d{1,2})', expand=False)
print(df.columns)
print(df[['Transaction unique identifier', 'postcode']].head())
def generate_edges(group_col):
    edges = []
    for _, group in df.groupby(group_col):
        postcodes = group['postcode'].unique()
        edges.extend(combinations(postcodes, 2))
    return edges
town_edges = generate_edges('Town/City')
county_edges = generate_edges('County')
all_edges = set(town_edges + county_edges)
edges_df = pd.DataFrame(list(all_edges), columns=['src', 'dst'])

Index(['Transaction unique identifier', 'Price', 'Date of Transfer',
       'Property Type', 'Old/New', 'Duration', 'Town/City', 'District',
       'County', 'PPDCategory Type', 'Record Status - monthly file only',
       'log_price', 'postcode'],
      dtype='object')
                   Transaction unique identifier postcode
13893637  {6146E264-E0D9-4C53-ACC8-48DB3954F80B}      E26
18522801  {26EBD75A-D90F-411C-85E5-4D56F0F66484}     BD75
7259339   {E700C723-9426-4924-8D3F-1730EC3B2BCC}      E70
2129082   {677E0E46-8E8F-4560-AD93-07F72D5AE6D5}       E0
3112615   {E2387F76-24EC-4A7E-8A27-220E500F0DC2}      E23
